In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from openpyxl import load_workbook
from openpyxl.styles import Font
from datetime import datetime

In [2]:
datapath = Path("./data")

In [3]:
df = pd.read_excel(datapath / "hmc_sap.xlsx")

In [4]:
df.head()

,회사구분,투자구분,프로젝트 정의,Project 명,WBS 요소,WBS 명,시스템 상태,사용자 상태,집행구분,통화,...,추진형태,투자계정,계획계정,예산담당자,연구과제,시작예정일,종료예정일,회계단위,설치장소,자본화일자
0,H,P,P-110150,상용 G엔진 국내 '15년 법규대응 엔진개발,P-110150.0003,시험부품구입비,REL,CLSE,마감,KRW,...,01:자체개발,T4:기술개발(시험),98601011.0,서명원/책임연구원,P-110150,2011-12-07,2015-12-31,H400,YPAB-5:상용전동화PT기능시험팀,2025-12-31
1,H,P,P-110150,상용 G엔진 국내 '15년 법규대응 엔진개발,P-110150.0004,공용부품 불출비,REL,CLSE,마감,KRW,...,01:자체개발,T4:기술개발(시험),98601011.0,서명원/책임연구원,P-110150,2011-12-07,2015-12-31,H400,YPAB-5:상용전동화PT기능시험팀,2025-12-31
2,H,P,P-110150,상용 G엔진 국내 '15년 법규대응 엔진개발,P-110150.0005,엔진불출비,REL,CLSE,마감,KRW,...,01:자체개발,T4:기술개발(시험),98601011.0,서명원/책임연구원,P-110150,2011-12-07,2015-12-31,H400,YPAB-5:상용전동화PT기능시험팀,2025-12-31
3,H,P,P-110150,상용 G엔진 국내 '15년 법규대응 엔진개발,P-110150.0006,연료비,REL,CLSE,마감,KRW,...,01:자체개발,T4:기술개발(시험),98601011.0,서명원/책임연구원,P-110150,2011-12-07,2015-12-31,H400,YPAB-5:상용전동화PT기능시험팀,2025-12-31
4,H,P,P-110150,상용 G엔진 국내 '15년 법규대응 엔진개발,P-110150.0007,해외출장비,REL,CLSE,마감,KRW,...,01:자체개발,T4:기술개발(시험),98601011.0,서명원/책임연구원,P-110150,2011-12-07,2015-12-31,H400,YPAB-5:상용전동화PT기능시험팀,2025-12-31


In [5]:
df.columns = df.columns.map(lambda x: x.replace(" ", ""))
df.columns

Index(['회사구분', '투자구분', '프로젝트정의', 'Project명', 'WBS요소', 'WBS명', '시스템상태', '사용자상태',
       '집행구분', '통화', '부대비포함', 'WBS통화키', '청구누적', '최초품의', '최종품의', '전체예산', '전체실적',
       '전체약정', '2025년최초계획', '당년운영금액', '당년가용예산', '2025년계획', '2025년예산',
       '2025년실적', '2025년약정', '2026년이후계획', '2026년이후예산', '2026년이후약정', '책임코스트센터',
       '총괄팀', '통제팀', '투자지역', '레벨텍스트1', '레벨텍스트2', '레벨텍스트3', '레벨텍스트4', '집행유형',
       '추진형태', '투자계정', '계획계정', '예산담당자', '연구과제', '시작예정일', '종료예정일', '회계단위',
       '설치장소', '자본화일자'],
      dtype='object')

In [6]:
df.dropna(subset="프로젝트정의", inplace=True)
df.shape

(1314, 47)

In [7]:
df["집행구분"].value_counts()

집행구분
마감      1146
미승인       87
승인        67
종료        10
부분승인       4
Name: count, dtype: int64

In [8]:
df = df[df["집행구분"] != "마감"]
df.shape

(168, 47)

In [9]:
df["프로젝트정의"].isna().sum()

np.int64(0)

In [10]:
budget_curr_op = df["당년운영금액"].sum()
print(f"{budget_curr_op:>15,} KRW")

  9,373,788,770 KRW


In [11]:
df.groupby("투자구분")["당년운영금액"].sum().apply(lambda x: "{:,} KRW".format(x))

투자구분
B      460,998,139 KRW
P    3,136,743,455 KRW
T    5,776,047,176 KRW
Name: 당년운영금액, dtype: object

In [12]:
df_real = pd.read_csv(datapath / "manager.txt", delimiter="\t")
df_real.head()

,프로젝트정의,실질담당자
0,P-180116,이중희 책임
1,P-190136,이성학 책임
2,P-220012,이중희 책임
3,P-220214,이중희 책임
4,P-220216,이성학 책임


In [13]:
def analyze_budget(df, mode=None):
    cond = (df["투자구분"] == mode) & (df["당년운영금액"] > 0)
    curr_op_budget = df.loc[cond, ["프로젝트정의", "Project명", "WBS요소", "WBS명", "집행구분", "당년운영금액", '당년가용예산', 
                                   '2025년실적', '2025년약정', '2026년이후계획']]
    curr_op_budget = pd.merge(curr_op_budget, df_real, on="프로젝트정의", how="left")
    cond = (curr_op_budget["집행구분"] == "미승인") & (curr_op_budget["당년운영금액"] > 0)
    curr_op_budget["미품의"] = curr_op_budget.loc[cond, ["당년운영금액"]]

    cob = curr_op_budget.sum(numeric_only=True)
    total_check = cob['당년가용예산'] + cob['2025년실적'] + cob['2025년약정'] + cob['미품의']
    total_check_info = f"집행 완료: {round((cob['2025년실적'] + cob['2025년약정']) / 1e8, 1)}억, "
    total_check_info += f"품의완/미집행: {round(cob['당년가용예산'] / 1e8, 1)}억, "
    total_check_info += f"미품의: {round(cob['미품의'] / 1e8, 1)}억, "
    total_check_info += f"총계: {round(total_check / 1e8, 1)}억원"
    
    cond = (curr_op_budget["집행구분"] == "미승인") & (curr_op_budget["당년운영금액"] > 0)
    manager_df = curr_op_budget.loc[cond, ["프로젝트정의", "Project명", "실질담당자"]]
    manager_df = manager_df.drop_duplicates(subset="프로젝트정의")
    print(manager_df)
    
    return curr_op_budget, total_check_info, manager_df

In [14]:
def save_to_excel(filename, sheet_name, isfirst=True, curr_op_budget=None, total_check_info=None, manager_df=None):
    if isfirst:
        with pd.ExcelWriter(filename, engine="openpyxl") as writer:
            curr_op_budget.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)
    else:
        with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
            curr_op_budget.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

    book = load_workbook(filename=filename)
    sheet = book[sheet_name]
    
    last_row = sheet.max_row
    while last_row > 0 and all(cell.value is None for cell in sheet[last_row]):
        last_row -= 1
    
    col_widths = [10, 50, 12, 42, 8, 12, 12, 12, 12, 12, 12, 12]
    for i, col in enumerate(sheet.columns):
        sheet.column_dimensions[col[0].column_letter].width = col_widths[i]
    
    for col in ["D", "E", "F", "G", "H", "I", "J", "K", "L"]:
        for row in range(2, sheet.max_row + 1):
            cell = sheet[f"{col}{row}"]
            cell.number_format = "#,##0"
    
    sheet[f"A{last_row + 2}"] = "■ 투자 현황"
    sheet[f"A{last_row + 3}"] = f"{total_check_info}"
    sheet[f"A{last_row + 5}"] = "■ 미품의 항목"
    book.save(filename)

    with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
        manager_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=last_row+5)

    book = load_workbook(filename=filename)
    sheet = book[sheet_name]
    sans_font = Font(name="맑은 고딕", size=10, bold=False)
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = sans_font if cell.value is not None else None
    book.save(filename)

In [15]:
time = datetime.now().strftime("%y%m%d")
filename = datapath / f"{time} 예산현황(원화기준).xlsx"

sheet_name = "시설투자(B)"
curr_op_budget, total_check_info, manager_df = analyze_budget(df, mode="B")
save_to_excel(filename, sheet_name, True, curr_op_budget, total_check_info, manager_df)
sheet_name = "기술개발(T)"
curr_op_budget, total_check_info, manager_df = analyze_budget(df, mode="T")
save_to_excel(filename, sheet_name, False, curr_op_budget, total_check_info, manager_df)
sheet_name = "프로젝트(P)"
curr_op_budget, total_check_info, manager_df = analyze_budget(df, mode="P")
save_to_excel(filename, sheet_name, False, curr_op_budget, total_check_info, manager_df)

      프로젝트정의                            Project명   실질담당자
0   R-240660    [신규] 오일희석 개발용 실시간 오일 점도 측정 장비 구매  이대희 책임
6   R-250771             [노후교체] 라이너 내주면 스캐너 노후교체  이용수 책임
8   R-250966                  실시간 모니터링 VMU 업그레이드  정원택 연구
9   R-251310            [장비-추가] 실시간 오일 점도 측정기 구매  이대희 책임
10  R-257419  [장비-신규] 상용디젤엔진 유로7 보쉬 ECU 통신 모듈 구매  이성학 책임
      프로젝트정의                               Project명    실질담당자
25  R-251553             안전사고 예방을 위한 시험실 환경 개선 (용역)   이주광 책임
34  R-251877                 흡기 부압별 실시간 오일소모 측정 용역비  이동복 파트장
35  R-251903                    HL 유로6d 필드 엔진 회수 평가   김서영 책임
38  R-252114         원가절감 부품 적용을 위한 상용 엔진 신뢰성 개발(L)   하창훈 책임
39  R-252123         원가절감 부품 적용을 위한 상용 엔진 신뢰성 개발(H)   하창훈 책임
41  R-252798          원가절감 부품 적용을 위한 신뢰성 평가 (G-ENG)   이성학 책임
42  R-254700                        F HEV 선행 개발 용역비   정원택 연구
43  R-256793  A_E_F-엔진 필드및 양산엔진 품질 향상을 위한 부품 신뢰성 확보   정원택 연구
      프로젝트정의                  Project명   실질담당자
2   P-220214  상용차 F-엔진 국내 유로7 법규 대응 개발  이중희 책임
7   P-220216  상

In [16]:
fuel_budget = pd.read_csv(datapath / "fuel_budget.txt", delimiter="\t", index_col=None)
fuel_budget["프로젝트정의"] = fuel_budget["WBS요소"].astype(str).apply(lambda x: x[:8])
fuel_budget = pd.merge(fuel_budget, df_real, on="프로젝트정의", how="left")
fuel_budget

,WBS요소,전액,디젤,연료비(디젤),연료비(CNG),우선순위,프로젝트정의,실질담당자
0,R-241851.0001,0,1,4200000,0,1,R-241851,이동복 파트장
1,R-252536.0001,0,1,120000000,0,2,R-252536,우형철 책임
2,R-251723.0004,1,1,0,0,3,R-251723,이대희 책임
3,R-242047.0001,1,1,0,0,4,R-242047,최해근 연구
4,R-242335.0001,1,1,0,0,5,R-242335,우형철 책임
5,R-240948.0001,1,1,0,0,6,R-240948,김주호 책임
6,R-240630.0001,1,1,0,0,7,R-240630,김주호 책임
7,P-250025.0033,1,1,0,0,8,P-250025,조대훈 책임
8,R-232139.0001,1,0,0,0,9,R-232139,김주환 책임


In [17]:
fuel_budget1 = pd.merge(fuel_budget, df[["WBS요소", "당년가용예산"]], on="WBS요소", how="left")
cond = (fuel_budget1["전액"] == 1) & (fuel_budget1["디젤"] == 1)
fuel_budget1.loc[cond, "연료비(디젤)"] = fuel_budget1["당년가용예산"]
cond = (fuel_budget1["전액"] == 1) & (fuel_budget1["디젤"] == 0)
fuel_budget1.loc[cond, "연료비(CNG)"] = fuel_budget1["당년가용예산"]
fuel_budget1

,WBS요소,전액,디젤,연료비(디젤),연료비(CNG),우선순위,프로젝트정의,실질담당자,당년가용예산
0,R-241851.0001,0,1,4200000,0,1,R-241851,이동복 파트장,30340768
1,R-252536.0001,0,1,120000000,0,2,R-252536,우형철 책임,150000000
2,R-251723.0004,1,1,149869500,0,3,R-251723,이대희 책임,149869500
3,R-242047.0001,1,1,175435801,0,4,R-242047,최해근 연구,175435801
4,R-242335.0001,1,1,200091000,0,5,R-242335,우형철 책임,200091000
5,R-240948.0001,1,1,317990436,0,6,R-240948,김주호 책임,317990436
6,R-240630.0001,1,1,383486293,0,7,R-240630,김주호 책임,383486293
7,P-250025.0033,1,1,59700000,0,8,P-250025,조대훈 책임,59700000
8,R-232139.0001,1,0,0,21930504,9,R-232139,김주환 책임,21930504


In [18]:
fuel_budget1["연료비(디젤)"].sum(), fuel_budget1["연료비(CNG)"].sum()

(np.int64(1410773030), np.int64(21930504))

In [19]:
sheet_name = "연료비(디젤,CNG)"
with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    fuel_budget1.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

book = load_workbook(filename=filename)
sheet = book[sheet_name]

last_row = sheet.max_row
while last_row > 0 and all(cell.value is None for cell in sheet[last_row]):
    last_row -= 1

col_widths = [16, 4, 4, 12, 12, 12, 12, 12, 12]
for i, col in enumerate(sheet.columns):
    sheet.column_dimensions[col[0].column_letter].width = col_widths[i]

for col in ["D", "E", "I"]:
    for row in range(2, sheet.max_row + 1):
        cell = sheet[f"{col}{row}"]
        cell.number_format = "#,##0"

sheet[f"A{last_row + 2}"] = "■ 연료비 현황"
fuel_info = f"연료비(디젤): {round(fuel_budget1['연료비(디젤)'].sum() / 1e8, 2):,}억, "
fuel_info += f"연료비(CNG): {round(fuel_budget1['연료비(CNG)'].sum() / 1e8, 2):,}억원"
sheet[f"A{last_row + 3}"] = fuel_info

sans_font = Font(name="맑은 고딕", size=10, bold=False)
for row in sheet.iter_rows():
    for cell in row:
        cell.font = sans_font if cell.value is not None else None

book.save(filename)